# The matrix formulation

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [The matrix formulation](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb

import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

from EconModel import jit
from ConSavModel import ConSavModelClass

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = ConSavModelClass()

In [3]:
par = model.par
sol = model.sol
sim = model.sim
D = sim.D_
Dbeg = sim.Dbeg_

In [4]:
model.solve(algo='egm')
model.prepare_simulate(algo='hist')
model.simulate_hist_alt()

iteration    0 solved in 5.5 secs   [max abs. diff. 4.95e+01]
iteration    1 solved in 0.0 secs   [max abs. diff. 1.67e+01]
iteration    2 solved in 0.0 secs   [max abs. diff. 8.37e+00]
iteration    3 solved in 0.0 secs   [max abs. diff. 5.04e+00]
iteration    4 solved in 0.0 secs   [max abs. diff. 3.38e+00]
iteration    5 solved in 0.0 secs   [max abs. diff. 2.42e+00]
iteration    6 solved in 0.0 secs   [max abs. diff. 1.82e+00]
iteration    7 solved in 0.0 secs   [max abs. diff. 1.42e+00]
iteration    8 solved in 0.0 secs   [max abs. diff. 1.14e+00]
iteration    9 solved in 0.0 secs   [max abs. diff. 9.40e-01]
iteration  100 solved in 0.0 secs   [max abs. diff. 4.07e-03]
iteration  200 solved in 0.0 secs   [max abs. diff. 1.38e-05]
iteration  300 solved in 0.0 secs   [max abs. diff. 1.28e-08]
iteration  304 solved in 0.0 secs   [max abs. diff. 9.67e-09]
model solved in 5.7 secs
model prepared for simulation in 0.0 secs
model simulated in 6.6 secs [430 iterations]


## 2. <a id='toc2_'></a>[The matrix formulation](#toc0_)

**The stochastic transition, $\boldsymbol{D}_{t} =\Pi_{z}^{\prime}\underline{\boldsymbol{D}}_{t}$:**

In [5]:
@nb.njit
def build_Pi_z(par):

    Pi_z = np.zeros((par.Nz*par.Na,par.Nz*par.Na))

    i = 0
    for i_z_lag in range(par.Nz):
        for i_a_lag in range(par.Na):
            j = 0
            for i_z in range(par.Nz):
                for i_a_lag_ in range(par.Na):
                    if i_a_lag == i_a_lag_: # does not change
                        Pi_z[i,j] = par.z_trans[i_z_lag,i_z]                     
                    else:
                        Pi_z[i,j] = 0.0
                    j += 1
            i += 1
    
    return Pi_z
            

In [6]:
with jit(model) as model_jit:
    Pi_z = build_Pi_z(model_jit.par)

In [7]:
D_alt = Pi_z.T@Dbeg.ravel()
max_abs_diff = np.max(np.abs(D_alt-D.ravel()))
assert np.isclose(max_abs_diff,0.0)

**The choice transition, $\underline{\boldsymbol{D}}_{t+1} =\Lambda_{t}^{\prime}\boldsymbol{D}_{t}$:**

In [8]:
@nb.njit
def build_Lambda(par,sol):

    Lambda = np.zeros((par.Nz*par.Na,par.Nz*par.Na))

    i = 0
    for i_z in range(par.Nz):
        for i_a_lag in range(par.Na):
            j = 0
            for i_z_ in range(par.Nz):
                for i_a in range(par.Na):
                    if i_z == i_z_ and i_a == sol.pol_indices[i_z,i_a_lag]:
                        Lambda[i,j] = sol.pol_weights[i_z,i_a_lag]
                    elif i_z == i_z_ and i_a == sol.pol_indices[i_z,i_a_lag]+1:
                        Lambda[i,j] = 1-sol.pol_weights[i_z,i_a_lag]
                    else:
                        Lambda[i,j] = 0.0
                    j += 1
            i += 1
     
    return Lambda

In [9]:
with jit(model) as model_jit:
    Lambda = build_Lambda(model_jit.par,model_jit.sol)

In [10]:
Dbeg_plus_alt = Lambda.T@D.ravel()
max_abs_diff = np.max(np.abs(Dbeg_plus_alt-Dbeg.ravel()))
assert np.isclose(max_abs_diff,0.0)